In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas as pd
from tqdm import tqdm

# Load the dataset
df = pd.read_csv('file3.csv')

# Print number of entries belonging to each class in the original dataset
class_counts = df['Sentiment_Class'].value_counts()
print("Number of entries belonging to each class in the original dataset:")
for sentiment_class, count in class_counts.items():
    print(f"{sentiment_class}: {count}")

# Convert Sentiment Classes to Integers
label_mapping = {'Negative': 0, 'Nuetral': 1, 'Positive': 2, 'Mixed_feelings': 3, 'Not_relevant': 4}
df['Sentiment_Class'] = df['Sentiment_Class'].map(label_mapping)

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(label_mapping))

# Tokenize and encode the dataset
encoded_data = tokenizer.batch_encode_plus(
    df['commentText'].tolist(),
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    max_length=256,
    return_tensors='pt',
    truncation=True
)

# Extract input tensors
input_ids = encoded_data['input_ids']
attention_mask = encoded_data['attention_mask']
labels = torch.tensor(df['Sentiment_Class'].values)

# Split the dataset
train_inputs, test_inputs, train_labels, test_labels, train_masks, test_masks = train_test_split(
    input_ids, labels, attention_mask, random_state=42, test_size=0.2
)

# Create DataLoader for train and test sets
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=False)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_dataloader) * 2
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Train the model with early stopping based on validation loss
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

epochs = 2
best_val_loss = float('inf')
patience = 0
for epoch in range(epochs):
    # Training loop with progress bar
    model.train()
    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False)
    for batch in progress_bar:
        inputs, masks, labels = batch
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=masks, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Update progress bar
        progress_bar.set_postfix({'training_loss': f'{loss.item():.3f}'})

    # Validation
    model.eval()
    val_losses = []
    for batch in test_dataloader:
        inputs, masks, labels = batch
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(inputs, attention_mask=masks, labels=labels)
            val_loss = outputs.loss.item()
            val_losses.append(val_loss)

    avg_val_loss = sum(val_losses) / len(val_losses)
    print(f'Epoch {epoch + 1}/{epochs}, Test Loss: {avg_val_loss:.4f}')

    # Early stopping based on validation loss
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience = 0
    else:
        patience += 1
        if patience > 2:
            print("Early stopping triggered, no improvement in test loss.")
            break

# Initialize empty list to store predictions
test_predictions = []

# Make predictions on the test set
for batch in test_dataloader:
    inputs, masks, labels = batch
    inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(inputs, attention_mask=masks)
        _, predicted_labels = torch.max(outputs.logits, dim=1)
        test_predictions.extend(predicted_labels.cpu().numpy())

# Calculate accuracy and F1 score
accuracy = accuracy_score(test_labels.cpu().numpy(), test_predictions)
f1 = f1_score(test_labels.cpu().numpy(), test_predictions, average='weighted')

# Print classification report, accuracy, and F1 score
print("Classification Report:")
print(classification_report(test_labels.cpu().numpy(), test_predictions, target_names=label_mapping.keys()))
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

# Save the model using torch.save()
output_dir = './bert_sentiment_model'
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'tokenizer': tokenizer,
    'label_mapping': label_mapping
}, output_dir)

print("Model saved successfully at:", output_dir)


Number of entries belonging to each class in the original dataset:
Not_relevant: 4127
Nuetral: 4072
Positive: 1716
Negative: 1565
Mixed_feelings: 352


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\Acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/2, Test Loss: 1.4756


Epoch 2/2, Test Loss: 1.1119
Classification Report:
                precision    recall  f1-score   support

      Negative       0.00      0.00      0.00       276
       Nuetral       0.70      0.81      0.75       823
      Positive       0.00      0.00      0.00       346
Mixed_feelings       0.00      0.00      0.00        72
  Not_relevant       0.48      0.80      0.60       850

      accuracy                           0.57      2367
     macro avg       0.24      0.32      0.27      2367
  weighted avg       0.42      0.57      0.48      2367

Accuracy: 0.5699
F1 Score: 0.4776


C:\Users\Acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Acer\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: U

RuntimeError: File ./bert_sentiment_model cannot be opened.

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas as pd
from tqdm import tqdm

# Load the dataset
df = pd.read_csv('file3.csv')

# Print number of entries belonging to each class in the original dataset
class_counts = df['Sentiment_Class'].value_counts()
print("Number of entries belonging to each class in the original dataset:")
for sentiment_class, count in class_counts.items():
    print(f"{sentiment_class}: {count}")

# Convert Sentiment Classes to Integers
label_mapping = {'Negative': 0, 'Nuetral': 1, 'Positive': 2, 'Mixed_feelings': 3, 'Not_relevant': 4}
df['Sentiment_Class'] = df['Sentiment_Class'].map(label_mapping)

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(label_mapping))

# Tokenize and encode the dataset
encoded_data = tokenizer.batch_encode_plus(
    df['commentText'].tolist(),
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    max_length=256,
    return_tensors='pt',
    truncation=True
)

# Extract input tensors
input_ids = encoded_data['input_ids']
attention_mask = encoded_data['attention_mask']
labels = torch.tensor(df['Sentiment_Class'].values)

# Split the dataset
train_inputs, test_inputs, train_labels, test_labels, train_masks, test_masks = train_test_split(
    input_ids, labels, attention_mask, random_state=42, test_size=0.2
)

# Print number of entries belonging to each class in the testing split
test_class_counts = test_labels.bincount()
print("\nNumber of entries belonging to each class in the testing split:")
for sentiment_class, count in enumerate(test_class_counts):
    print(f"{sentiment_class}: {count}")

# Create DataLoader for train and test sets
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=False)

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_dataloader) * 2
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Train the model with early stopping based on validation loss
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

epochs = 2
best_val_loss = float('inf')
patience = 0
for epoch in range(epochs):
    # Training loop with progress bar
    model.train()
    progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False)
    for batch in progress_bar:
        inputs, masks, labels = batch
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=masks, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        # Update progress bar
        progress_bar.set_postfix({'training_loss': f'{loss.item():.3f}'})

    # Validation
    model.eval()
    val_losses = []
    for batch in test_dataloader:
        inputs, masks, labels = batch
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(inputs, attention_mask=masks, labels=labels)
            val_loss = outputs.loss.item()
            val_losses.append(val_loss)

    avg_val_loss = sum(val_losses) / len(val_losses)
    print(f'Epoch {epoch + 1}/{epochs}, Test Loss: {avg_val_loss:.4f}')

    # Early stopping based on validation loss
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience = 0
    else:
        patience += 1
        if patience > 2:
            print("Early stopping triggered, no improvement in test loss.")
            break

# Initialize lists to store predictions and true labels
all_predictions = []
all_true_labels = []

# Evaluate the model on the test set
model.eval()
for batch in test_dataloader:
    inputs, masks, labels = batch
    inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

    with torch.no_grad():
        outputs = model(inputs, attention_mask=masks)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

        # Store predictions and true labels
        all_predictions.extend(predictions.cpu().numpy())
        all_true_labels.extend(labels.cpu().numpy())

# Calculate accuracy and F1 score
accuracy = accuracy_score(all_true_labels, all_predictions)
f1 = f1_score(all_true_labels, all_predictions, average='weighted')

# Print accuracy and F1 score
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')

# Print classification report
print("Classification Report:")
print(classification_report(all_true_labels, all_predictions, target_names=label_mapping.keys()))

# Save the model using torch.save()
output_dir = './bert_sentiment_model'
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'scheduler_state_dict': scheduler.state_dict(),
    'tokenizer': tokenizer,
    'label_mapping': label_mapping
}, output_dir)

print("Model saved successfully at:", output_dir)
